<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [46]</a>'.</span>

# pa005: High Value Customer Identification (Insiders)

# 0.0. Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

from IPython.display         import Image

from matplotlib              import pyplot       as plt
from matplotlib              import gridspec
from plotly                  import express      as px
from IPython.display         import HTML

import hdbscan
import umap.umap_                                 as umap

from sklearn                 import cluster       as c
from sklearn                 import metrics       as m
from sklearn                 import preprocessing as pp
from sklearn                 import decomposition as dd
from sklearn.manifold        import TSNE
from sklearn.ensemble        import RandomForestRegressor
from sklearn.neighbors       import NearestNeighbors
from sklearn                 import mixture       as mx
from scipy.cluster           import hierarchy     as hc


from yellowbrick.cluster     import KElbowVisualizer, SilhouetteVisualizer

import sqlite3
from sqlalchemy              import create_engine

import warnings
warnings.filterwarnings('ignore')

## 0.1 Helper Functions

In [2]:
def get_silhouettes( data, labels ):
    
    silhouette_scores = []

    for label in labels:

        # model performance
            sil = m.silhouette_score( data, label, metric = 'euclidean' )
            silhouette_scores.append( np.round(sil,2) )
    
    return silhouette_scores

def plot_clusters( data, k, labels ):
    
    data = data.copy()
    data.columns = ['embedding_x', 'embedding_y']
    
    nrows = len(k) // 4 if len(k) % 4 == 0 else (len(k)//4) + 1
    fig = plt.figure(figsize = (21,18) ) 
    specs = gridspec.GridSpec( ncols = 4, nrows = nrows, figure = fig, wspace = .3, hspace = .3 )
    
    plots = []
    
    silhouettes = get_silhouettes( data, labels)
    
    for i in range(nrows):
        for j in range(4):
            ax = fig.add_subplot( specs[ i, j ] )
            plots.append( ax )
    
    for i in range(len( k )):
       
        # model evaluation
        sil = silhouettes[i]
        
        data['cluster'] = labels[i]
        
        sns.scatterplot(data = data, x='embedding_x',y='embedding_y', 
                        hue = 'cluster', palette = 'deep', 
                        ax = plots[i], legend = False)
        
        
        title = "Num. Clusters = " + str(k[i]) + " | Silhouette Score: " + str(np.round(sil,2))
        plots[i].set_title( title )
        plots[i].set_xlabel(xlabel = "")
        plots[i].set_ylabel(ylabel = "")

        
def best_silhouette( data, k, labels ):
    
    silhouettes = get_silhouettes( data = data, labels = labels )

    plt.figure( figsize = ( 10, 6 ))
    plt.plot( k, silhouettes, linestyle = '--', marker = 'o', color = 'b')
    plt.xlabel( 'K' );
    plt.ylabel( 'Silhouette Score' );
    plt.title( 'Silhouette Score x K' );
    

def plot_cluster_silhouette(X, labels, k):
    
    fig, ax = plt.subplots( 1, 1 )
    fig.set_size_inches( 8, 7 )

    ax.set_xlim( [ -0.1, 1 ] )
    ax.set_ylim( [ 0, len( X ) + ( k + 1 ) * 10] )
    
    samples_silhouette_values = m.silhouette_samples( X, labels )

    y_lower = 10
    for i in range( k ):

        # select clusters
        ith_samples_silhouette_values = samples_silhouette_values[ labels == i]
        
        # sort values
        ith_samples_silhouette_values.sort()

        # size clusters
        size_cluster_i = ith_samples_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        cmap = plt.get_cmap( 'Spectral' )
        color = cmap( i / k )

        ax.fill_betweenx( np.arange( y_lower, y_upper ), 0, ith_samples_silhouette_values  )
        
        y_lower = y_upper + 10

    ax.set_yticks([])
    ax.set_xticks( [-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.legend( np.unique(labels) )
    ax.set_title(" Clusters Silhouettes")

## 0.2 Loading Dataset

In [3]:
# Load data
path = "/home/igor/Documents/repos/ds_em_clusterizacao/project/"
ecommerce = pd.read_csv(path + 'data/Ecommerce.csv',encoding='latin1')

# Drop extra column
ecommerce = ecommerce.iloc[:,:-1]

# 1.0. Data description

In [4]:
ecommerce1 = ecommerce.copy()

## 1.1. Rename Columns

In [5]:
new_cols =  ['invoice_no', 'stock_code','description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
ecommerce1.columns = new_cols

## 1.2. Data Dimensions

In [6]:
ecommerce1.shape

(541909, 8)

## 1.3. Data types

In [7]:
ecommerce1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [8]:
ecommerce1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [9]:
# Missing values on customer ID
ecommerce_missing = ecommerce1[ecommerce1['customer_id'].isna()]
ecommerce_not_missing = ecommerce1[~ecommerce1['customer_id'].isna()]

In [10]:
ecommerce_missing_customerid = ecommerce_missing[['invoice_no','customer_id']].drop_duplicates()
ecommerce_missing_customerid['customer_id'] = np.arange( 19000, 19000 + len(ecommerce_missing_customerid), 1 )
ecommerce_missing = pd.merge(ecommerce_missing.drop('customer_id', axis = 1), ecommerce_missing_customerid, on = 'invoice_no', how = 'left')
ecommerce_missing = ecommerce_missing[ecommerce_not_missing.columns]
ecommerce1 = pd.concat([ecommerce_missing, ecommerce_not_missing])

In [11]:
ecommerce1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
customer_id        0
country            0
dtype: int64

## 1.6. Change Dtypes

In [12]:
# Invoice date
ecommerce1['invoice_date'] = pd.to_datetime(ecommerce1['invoice_date'], format = '%d-%b-%y')

# Customer id
ecommerce1['customer_id'] = ecommerce1['customer_id'].astype(np.int64)

In [13]:
ecommerce1.sort_values('invoice_no').head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2016-11-29,4.25,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2016-11-29,7.65,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,17850,United Kingdom


## 1.7. Drop Duplicates

In [14]:
#ecommerce1 = ecommerce1.drop_duplicates()

## 1.8. Descriptive Statistics

In [15]:
num_attributes = ecommerce1.select_dtypes( include = ['int64', 'float64'])
cat_attributes = ecommerce1.select_dtypes( exclude = ['int64', 'float64', 'datetime64[ns]'])

### 1.8.1. Numerical Attributes

In [16]:
# Central tendency (mean, median)
ct1 = ( num_attributes.apply( np.mean ) ).T
ct2 = ( num_attributes.apply( np.median ) ).T
# Dispersion ( std, min, max. range, skew, kurtosis)
d1 = ( num_attributes.apply( np.std ) ).T
d2 = ( num_attributes.apply( np.min ) ).T
d3 = ( num_attributes.apply( np.max ) ).T
d4 = ( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = ( num_attributes.apply( lambda x: x.max() - x.skew() ) ).T
d6 = ( num_attributes.apply( lambda x: x.max() - x.kurtosis() ) ).T

# Concatenate
ecommerce_stats = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6], axis = 1 ).reset_index()
ecommerce_stats.columns = ['attributes','min','max', 'range', 'mean', 'median',' std',' skew', 'kurtosis']
ecommerce_stats

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,80995.264076,-38774.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,38783.493028,-20035.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,22708.512551,22709.804287


### 1.8.2. Categorical Attributes

##### Invoice Number

In [17]:
 # Invoice number
invoices_w_letters = ecommerce1.loc[ecommerce1['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), : ]
negative_quantity = len( invoices_w_letters.loc[invoices_w_letters['quantity'] < 0, : ] )
print('Total number of invoices number with letter: ',len( invoices_w_letters ), '\nTotal number of negative quantity: ', negative_quantity)

Total number of invoices number with letter:  9291 
Total number of negative quantity:  9288


##### Stock code

In [18]:
# Stock code
# Check only characters on stock code
ecommerce1.loc[ecommerce1['stock_code'].apply( lambda x: bool( re.search( '^[a-zA-Z]+$',  x ) ) ), 'stock_code' ].unique()

# Action: Remove stock code in ['DOT', 'M', 'S', 'AMAZONFEE', 'm', 'POST', 'DCGSSBOY', 'DCGSSGIRL', 'B', 'D', 'PADS', 'CRUK']


array(['DOT', 'M', 'S', 'AMAZONFEE', 'm', 'POST', 'DCGSSBOY', 'DCGSSGIRL',
       'B', 'D', 'PADS', 'CRUK'], dtype=object)

##### Description

In [19]:
# Action: delete description

##### Country

In [20]:
ecommerce1['country'].unique()

array(['United Kingdom', 'EIRE', 'Bahrain', 'Israel', 'Hong Kong',
       'Unspecified', 'France', 'Switzerland', 'Portugal', 'Australia',
       'Netherlands', 'Germany', 'Norway', 'Spain', 'Poland', 'Italy',
       'Belgium', 'Lithuania', 'Japan', 'Iceland', 'Channel Islands',
       'Denmark', 'Cyprus', 'Sweden', 'Austria', 'Finland', 'Greece',
       'Singapore', 'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Brazil', 'USA', 'European Community',
       'Malta', 'RSA'], dtype=object)

In [21]:
ecommerce1['country'].value_counts(normalize = True)

United Kingdom          0.914320
Germany                 0.017521
France                  0.015790
EIRE                    0.015124
Spain                   0.004674
Netherlands             0.004375
Belgium                 0.003818
Switzerland             0.003694
Portugal                0.002803
Australia               0.002323
Norway                  0.002004
Italy                   0.001482
Channel Islands         0.001399
Finland                 0.001283
Cyprus                  0.001148
Sweden                  0.000853
Unspecified             0.000823
Austria                 0.000740
Denmark                 0.000718
Japan                   0.000661
Poland                  0.000629
Israel                  0.000548
USA                     0.000537
Hong Kong               0.000531
Singapore               0.000423
Iceland                 0.000336
Canada                  0.000279
Greece                  0.000269
Malta                   0.000234
United Arab Emirates    0.000125
European C

In [22]:
ecommerce1[['customer_id','country']].drop_duplicates().groupby('country').count().reset_index().sort_values(by = 'customer_id', ascending = False).head(4)

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44


# 2.0. Data Filtering

In [23]:
ecommerce2 = ecommerce1

In [24]:
# Drop description column
ecommerce2 = ecommerce2.drop('description', axis = 1)

In [25]:
# Categorical attributes

# Stock code
ecommerce2 = ecommerce2[~ecommerce2['stock_code'].isin(['DOT', 'M', 'S', 'AMAZONFEE', 'm', 'POST', 'DCGSSBOY', 'DCGSSGIRL', 'B', 'D', 'PADS', 'CRUK'])]

# Country
ecommerce2 = ecommerce2[~ecommerce2['country'].isin( ['European Community', 'Unspecified'] )] 

# Numerical attributes

# Unit price
ecommerce2 = ecommerce2[ecommerce2['unit_price'] >= 0.04]

# Bad users
ecommerce2 = ecommerce2[~ecommerce2['customer_id'].isin( [16446] )]

# Quantity
ecommerce_returns = ecommerce2.loc[ecommerce2['quantity'] < 0, : ] 
ecommerce_purchases = ecommerce2.loc[ecommerce2['quantity'] >= 0, : ] 

# 3.0. Feature Engineering

In [26]:
ecommerce3 = ecommerce2

In [27]:
customer_df = ecommerce3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis = 1).drop_duplicates(ignore_index = True)

## 3.1. Feature Creation

### 3.1.1. Gross Revenue

In [28]:
# Gross Revenue
ecommerce_purchases['gross_revenue'] = ecommerce_purchases['quantity'] * ecommerce_purchases['unit_price']
df_gross_revenue = ecommerce_purchases[['gross_revenue','customer_id']].groupby('customer_id').sum().reset_index()
customer_df = pd.merge(customer_df, df_gross_revenue,on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2. Recency

In [29]:
# Recency - Last day purchase
df_recency = ecommerce_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency'] = (ecommerce_purchases['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id','recency']].copy()

# Merge
customer_df = pd.merge(customer_df, df_recency, on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id       0
gross_revenue    91
recency          91
dtype: int64

### 3.1.3. Quantity of purchases

In [30]:
# qtty purchased
df_frequency = ecommerce_purchases[['invoice_no', 'customer_id']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns = {'invoice_no' : 'qtty_invoices'})

# Merge
customer_df = pd.merge(customer_df, df_frequency, on = 'customer_id', how = 'left')
customer_df.isna().sum()

customer_id       0
gross_revenue    91
recency          91
qtty_invoices    91
dtype: int64

### 3.1.4. Quantity of items purchased

In [31]:
# qtty purchased
df_products = ecommerce_purchases[['quantity', 'customer_id']].groupby('customer_id').sum().reset_index().rename(columns = {'quantity' : 'qtty_items'})

# Merge
customer_df = pd.merge(customer_df, df_products, on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id       0
gross_revenue    91
recency          91
qtty_invoices    91
qtty_items       91
dtype: int64

### 3.1.5. Quantity of products purchased

In [32]:
# qtty purchased
df_products = ecommerce_purchases[['stock_code', 'customer_id']].groupby('customer_id').count().reset_index().rename(columns = {'stock_code' : 'qtty_products'})

# Merge
customer_df = pd.merge(customer_df, df_products, on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id       0
gross_revenue    91
recency          91
qtty_invoices    91
qtty_items       91
qtty_products    91
dtype: int64

### 3.1.6. Average Ticket

In [33]:
# AVG Ticket
df_avgticket = ecommerce_purchases[['customer_id','gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns = {'gross_revenue':'avg_ticket'})

# Merge
customer_df = pd.merge(customer_df, df_avgticket, on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id       0
gross_revenue    91
recency          91
qtty_invoices    91
qtty_items       91
qtty_products    91
avg_ticket       91
dtype: int64

### 3.1.7. Average Recency Days

In [34]:
# AVG Recency days
aux_recency = ecommerce2[['customer_id','invoice_date']].drop_duplicates().sort_values( ['customer_id','invoice_date'], ascending = [True, True])
aux_recency['next_customer_id'] = aux_recency['customer_id'].shift()
aux_recency['previous_date'] = aux_recency['invoice_date'].shift()
aux_recency['avg_recency_days'] = aux_recency.apply( lambda x:( ( x['invoice_date'] - x['previous_date'] ).days if x['customer_id'] == x['next_customer_id'] else np.nan), axis = 1 )
aux_recency = aux_recency.drop( ['invoice_date','next_customer_id','previous_date'], axis = 1).dropna()
aux_recency = aux_recency.groupby('customer_id').mean().reset_index()

# Merge
customer_df = pd.merge(customer_df, aux_recency, on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id            0
gross_revenue         91
recency               91
qtty_invoices         91
qtty_items            91
qtty_products         91
avg_ticket            91
avg_recency_days    2816
dtype: int64

### 3.1.8. Fequecy Purchase

In [35]:
# Frquency purchase
frequency_df = (
                ecommerce2[['customer_id','invoice_no', 'invoice_date']].drop_duplicates()
                                                                        .groupby('customer_id')
                                                                        .agg( max_date = ('invoice_date', 'max'),
                                                                              min_date = ('invoice_date', 'min'),
                                                                              qtty_purchases = ('invoice_no', 'count')     
                                                                            ).reset_index()
              )
frequency_df['date_interval'] = ((frequency_df['max_date'] - frequency_df['min_date']).dt.days)
frequency_df['frequency'] = frequency_df.apply(lambda x: ( x['qtty_purchases'] / x['date_interval'] ) if x['date_interval'] != 0 else 0, axis = 1 )

# Merge
customer_df = pd.merge(customer_df, frequency_df[['customer_id','frequency']], on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id            0
gross_revenue         91
recency               91
qtty_invoices         91
qtty_items            91
qtty_products         91
avg_ticket            91
avg_recency_days    2816
frequency              0
dtype: int64

### 3.1.9. Returns

In [36]:
# Number of returns
customer_returns = ecommerce_returns[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns = {'quantity' : 'qtty_returns'})
customer_returns['qtty_returns'] = customer_returns['qtty_returns'] * - 1

# Merge
customer_df = pd.merge(customer_df, customer_returns, on = 'customer_id', how = 'left')
customer_df.loc[customer_df['qtty_returns'].isna(),'qtty_returns'] = 0

customer_df.isna().sum()

customer_id            0
gross_revenue         91
recency               91
qtty_invoices         91
qtty_items            91
qtty_products         91
avg_ticket            91
avg_recency_days    2816
frequency              0
qtty_returns           0
dtype: int64

### 3.1.10. Basket Size

In [37]:
basket_size = (ecommerce_purchases[['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                 .agg(invoices = ('invoice_no','nunique'),
                                                                      products = ('quantity','sum')).reset_index() 
                
)

basket_size['basket_size'] = basket_size['products'] / basket_size['invoices']

# Merge
customer_df = pd.merge(customer_df, basket_size[['customer_id','basket_size']], on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id            0
gross_revenue         91
recency               91
qtty_invoices         91
qtty_items            91
qtty_products         91
avg_ticket            91
avg_recency_days    2816
frequency              0
qtty_returns           0
basket_size           91
dtype: int64

### 3.1.11. Unique Basket Size

In [38]:
avg_basket_size = ( ecommerce_purchases[['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id')
                                                                 .agg( invoices = ('invoice_no','nunique'),
                                                                       products = ('stock_code','nunique') ).reset_index() 
                
                  )

avg_basket_size['unique_basket_size'] = avg_basket_size['products'] / avg_basket_size['invoices']
# Merge
customer_df = pd.merge(customer_df, avg_basket_size[['customer_id','unique_basket_size']], on = 'customer_id', how = 'left')

customer_df.isna().sum()

customer_id              0
gross_revenue           91
recency                 91
qtty_invoices           91
qtty_items              91
qtty_products           91
avg_ticket              91
avg_recency_days      2816
frequency                0
qtty_returns             0
basket_size             91
unique_basket_size      91
dtype: int64

# 4.0. Univariate & Bivariate Analysis

In [39]:
ecommerce4 = customer_df.dropna().copy()

# 5.0. Feature Selection

In [40]:
cols_select = ['customer_id','gross_revenue', 'recency', 'qtty_products', 'frequency', 'qtty_returns']
ecommerce4 = ecommerce4[cols_select].copy()

# 6.0. Data Preparation

In [41]:
ecommerce5 = ecommerce4.copy()

## 6.1. Rescaling

In [42]:
# Minmax Scaler
mm = pp.MinMaxScaler() # not normal distribution
ss = pp.StandardScaler() # normal distribution without outliers
rs = pp.RobustScaler() # normal distribution with outliers

ecommerce5['gross_revenue'] = mm.fit_transform(ecommerce5[['gross_revenue']])

ecommerce5['recency']       = mm.fit_transform(ecommerce5[['recency']])

ecommerce5['qtty_products'] = mm.fit_transform(ecommerce5[['qtty_products']])

ecommerce5['frequency']     = mm.fit_transform(ecommerce5[['frequency']])

ecommerce5['qtty_returns']  = mm.fit_transform(ecommerce5[['qtty_returns']])

# 7.0. Space Study

In [43]:
# Selected dataset
ecommerce6 = ecommerce5.copy()
X = ecommerce6.drop('customer_id', axis = 1)

## 7.4. Tree-Based Embedding

In [44]:
y = X['gross_revenue']
X = X.drop('gross_revenue', axis = 1)

In [45]:
# Model definition
rf_model = RandomForestRegressor( n_estimators = 100, random_state = 42 )

# Model training
rf_model.fit( X, y )

# Leaf
df_leaf = pd.DataFrame( rf_model.apply( X ) )

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [46]:
reducer = umap.UMAP( random_state = 42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_leaf['embedding_x'] = embedding[: , 0]
df_leaf['embedding_y'] = embedding[: , 1 ]

df_leaf.to_csv('/data/tree_based_embedding.csv', index = False)

OSError: Cannot save file into a non-existent directory: '/data'

# 9.0. Model Training

In [ ]:
ecommerce7 = df_leaf.iloc[:,-2:].copy()
ecommerce7['customer_id'] = ecommerce6['customer_id']
X = ecommerce7.iloc[:,:-1].copy()

## 9.1. GMM

In [ ]:
# Model definition
k = 10
gmm_model = mx.GaussianMixture( n_components = k, n_init = 30 )
# Model training
gmm_model.fit( X )

# Clustering
labels = gmm_model.predict( X )

In [ ]:
# Saving clusters labels
idx = np.argsort( gmm_model.means_.sum(axis = 1) )

new_cluster_label = np.zeros_like( idx )

clustar_name = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

new_cluster_label[idx] = clustar_name

labels = new_cluster_label[ labels ]

## 9.2. Cluster Validation

In [ ]:
## SS ( Silhouette Score)
print("SS value: {}".format ( m.silhouette_score(X, labels, metric = 'euclidean' ) ) )

# 10.0. Cluster Analysis

In [ ]:
ecommerce9 = ecommerce4.copy()
ecommerce9['cluster'] = labels

# Number of customers
no_customers = ecommerce9[['customer_id','cluster']].groupby('cluster').count().reset_index()
no_customers['percentage'] = ( no_customers['customer_id'] /  no_customers['customer_id'].sum() ) * 100

# AVG Gross Revenue
class_gross_revenue = ecommerce9[['gross_revenue','cluster']].groupby('cluster').mean().reset_index().rename(columns = {'gross_revenue':'avg_gross_revenue'})
df_cluster = pd.merge(no_customers, class_gross_revenue, how = 'inner', on = 'cluster')

# AVG Recency days
class_recency = ecommerce9[['recency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, class_recency, how = 'inner', on = 'cluster')

# AVG invoices
class_invoices = ecommerce9[['qtty_products','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, class_invoices, how = 'inner', on = 'cluster')

# Frequency
class_frequency = ecommerce9[['frequency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, class_frequency, how = 'inner', on = 'cluster')

# Returns
class_returns = ecommerce9[['qtty_returns','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, class_returns, how = 'inner', on = 'cluster')

# Percentual Gross Revenue
perc_gross_revenue = ecommerce9[['gross_revenue','cluster']].groupby('cluster').sum().reset_index()
perc_gross_revenue['percentual_gross_revenue'] = ( perc_gross_revenue['gross_revenue'] /  perc_gross_revenue['gross_revenue'].sum() ) * 100
df_cluster = pd.merge(df_cluster, perc_gross_revenue, how = 'inner', on = 'cluster')

In [ ]:
df_cluster

# 11.0. Deploy to Production

In [ ]:
ecommerce9.dtypes

In [ ]:
ecommerce9['recency'] = ecommerce9['recency'].astype( int )
ecommerce9['qtty_products'] = ecommerce9['qtty_products'].astype( int )
ecommerce9['qtty_returns'] = ecommerce9['qtty_returns'].astype( int )

## 11.1. Insert into SQLite

In [ ]:
# create table
query_create_table_insiders = """CREATE TABLE insiders (customer_id     INTEGER,
                                                       gross_revenue   REAL,
                                                       recency         INTEGER,
                                                       qtty_products   INTEGER,
                                                       frequency       REAL,
                                                       qtty_returns    INTEGER,
                                                       cluster         INTEGER)"""
    


connect = sqlite3.connect( 'insiders_db_sqlite' )
connect.execute( query_create_table_insiders )
connect.commit()
connect.close()

In [ ]:
# insert data
connect = create_engine('sqlite:///insiders_db_sqlite')
ecommerce9.to_sql('insiders', con = connect, if_exists = 'append', index = False)

In [ ]:
# # Consulting database

# connect = create_engine('sqlite:///insiders_db_sqlite')

# query = """
#         SELECT * FROM insiders
#         """
# df = pd.read_sql_query(query, connect)